# How to Convert Text to Vectors Using the Embedding API and Search for Similar Content

This document demonstrates how to convert text into vectors using the Embedding API and how to perform semantic searches for similar content.  
In the following example, we will vectorize 53 sample documents provided by Wikipedia and explore how to search for similar documents.  
Reference: [Azure OpenAI Embeddings Tutorial](https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings)


### Note
> - Due to a version conflict issue between pandas==2.0.3 and numpy==2.0.0, the pandas version has been updated to 2.1.2 as of July 2024.
> - Update December 2024: Switched to the text_embeddings_3_large API.

In [1]:
import os
import re
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv("DEPLOYMENT_NAME")
deployment_embedding_name = os.getenv("DEPLOYMENT_EMBEDDING_NAME")

Read the file (`./data/wiki_data.csv`) for vectorization and inspect it using pandas.

In [2]:
df_wiki_data=pd.read_csv(os.path.join(os.getcwd(),'data/wiki_data.csv'))
df_wiki_data

id                                                url  \
0     1            https://simple.wikipedia.org/wiki/April   
1     2           https://simple.wikipedia.org/wiki/August   
2     6              https://simple.wikipedia.org/wiki/Art   
3     8                https://simple.wikipedia.org/wiki/A   
4     9              https://simple.wikipedia.org/wiki/Air   
5    12  https://simple.wikipedia.org/wiki/Autonomous%2...   
6    13    https://simple.wikipedia.org/wiki/Alan%20Turing   
7    14  https://simple.wikipedia.org/wiki/Alanis%20Mor...   
8    17  https://simple.wikipedia.org/wiki/Adobe%20Illu...   
9    18        https://simple.wikipedia.org/wiki/Andouille   
10   19          https://simple.wikipedia.org/wiki/Farming   
11   21       https://simple.wikipedia.org/wiki/Arithmetic   
12   22         https://simple.wikipedia.org/wiki/Addition   
13   27        https://simple.wikipedia.org/wiki/Australia   
14   28  https://simple.wikipedia.org/wiki/American%20E...   
15   30      https://simple.wikipedia.org/wiki/Aquaculture   
16   32     https://simple.wikipedia.org/wiki/Abbreviation   
17   33            https://simple.wikipedia.org/wiki/Angel   
18   35     https://simple.wikipedia.org/wiki/Ad%20hominem   
19   37  https://simple.wikipedia.org/wiki/Native%20Ame...   
20   39            https://simple.wikipedia.org/wiki/Apple   
21   43  https://simple.wikipedia.org/wiki/Abrahamic%20...   
22   45          https://simple.wikipedia.org/wiki/Algebra   
23   47             https://simple.wikipedia.org/wiki/Atom   
24   48        https://simple.wikipedia.org/wiki/Astronomy   
25   49     https://simple.wikipedia.org/wiki/Architecture   
26   50          https://simple.wikipedia.org/wiki/Anatomy   
27   51         https://simple.wikipedia.org/wiki/Asteroid   
28   52      https://simple.wikipedia.org/wiki/Afghanistan   
29   53           https://simple.wikipedia.org/wiki/Angola   
30   54        https://simple.wikipedia.org/wiki/Argentina   
31   55          https://simple.wikipedia.org/wiki/Austria   
32   56          https://simple.wikipedia.org/wiki/Armenia   
33   57      https://simple.wikipedia.org/wiki/Archaeology   
34   58      https://simple.wikipedia.org/wiki/Application   
35   62           https://simple.wikipedia.org/wiki/Animal   
36   63     https://simple.wikipedia.org/wiki/Acceleration   
37   68  https://simple.wikipedia.org/wiki/Black%20pudding   
38   69    https://simple.wikipedia.org/wiki/Boot%20device   
39   70             https://simple.wikipedia.org/wiki/Boot   
40   71       https://simple.wikipedia.org/wiki/Bankruptcy   
41   74  https://simple.wikipedia.org/wiki/Breakfast%20...   
42   76          https://simple.wikipedia.org/wiki/Browser   
43   80       https://simple.wikipedia.org/wiki/Beekeeping   
44   83  https://simple.wikipedia.org/wiki/British%20En...   
45   85            https://simple.wikipedia.org/wiki/Being   
46   89          https://simple.wikipedia.org/wiki/Beijing   
47   90           https://simple.wikipedia.org/wiki/Bottle   
48   91            https://simple.wikipedia.org/wiki/Berry   
49   93             https://simple.wikipedia.org/wiki/Boil   
50   94            https://simple.wikipedia.org/wiki/Beard   
51   98            https://simple.wikipedia.org/wiki/Black   
52  100  https://simple.wikipedia.org/wiki/Bubonic%20pl...   

                              title  \
0                             April   
1                            August   
2                               Art   
3                                 A   
4                               Air   
5   Autonomous communities of Spain   
6                       Alan Turing   
7                 Alanis Morissette   
8                 Adobe Illustrator   
9                         Andouille   
10                          Farming   
11                       Arithmetic   
12                         Addition   
13                        Australia   
14                 American English   
15                      Aquacult

In [3]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_wiki_data['text']= df_wiki_data["text"].apply(lambda x : normalize_text(x))

### Check that the text in the documents does not exceed 8,192 tokens to use the Embedding API provided by Azure OpenAI.

In [4]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_wiki_data['n_tokens'] = df_wiki_data["text"].apply(lambda x: len(tokenizer.encode(x)))
df_wiki_data = df_wiki_data[df_wiki_data.n_tokens<8192]
len(df_wiki_data)
df_wiki_data

id                                                url  \
0     1            https://simple.wikipedia.org/wiki/April   
1     2           https://simple.wikipedia.org/wiki/August   
2     6              https://simple.wikipedia.org/wiki/Art   
3     8                https://simple.wikipedia.org/wiki/A   
4     9              https://simple.wikipedia.org/wiki/Air   
5    12  https://simple.wikipedia.org/wiki/Autonomous%2...   
6    13    https://simple.wikipedia.org/wiki/Alan%20Turing   
7    14  https://simple.wikipedia.org/wiki/Alanis%20Mor...   
8    17  https://simple.wikipedia.org/wiki/Adobe%20Illu...   
9    18        https://simple.wikipedia.org/wiki/Andouille   
10   19          https://simple.wikipedia.org/wiki/Farming   
11   21       https://simple.wikipedia.org/wiki/Arithmetic   
12   22         https://simple.wikipedia.org/wiki/Addition   
13   27        https://simple.wikipedia.org/wiki/Australia   
14   28  https://simple.wikipedia.org/wiki/American%20E...   
15   30      https://simple.wikipedia.org/wiki/Aquaculture   
16   32     https://simple.wikipedia.org/wiki/Abbreviation   
17   33            https://simple.wikipedia.org/wiki/Angel   
18   35     https://simple.wikipedia.org/wiki/Ad%20hominem   
19   37  https://simple.wikipedia.org/wiki/Native%20Ame...   
20   39            https://simple.wikipedia.org/wiki/Apple   
21   43  https://simple.wikipedia.org/wiki/Abrahamic%20...   
22   45          https://simple.wikipedia.org/wiki/Algebra   
23   47             https://simple.wikipedia.org/wiki/Atom   
24   48        https://simple.wikipedia.org/wiki/Astronomy   
25   49     https://simple.wikipedia.org/wiki/Architecture   
26   50          https://simple.wikipedia.org/wiki/Anatomy   
27   51         https://simple.wikipedia.org/wiki/Asteroid   
28   52      https://simple.wikipedia.org/wiki/Afghanistan   
29   53           https://simple.wikipedia.org/wiki/Angola   
30   54        https://simple.wikipedia.org/wiki/Argentina   
31   55          https://simple.wikipedia.org/wiki/Austria   
32   56          https://simple.wikipedia.org/wiki/Armenia   
33   57      https://simple.wikipedia.org/wiki/Archaeology   
34   58      https://simple.wikipedia.org/wiki/Application   
35   62           https://simple.wikipedia.org/wiki/Animal   
36   63     https://simple.wikipedia.org/wiki/Acceleration   
37   68  https://simple.wikipedia.org/wiki/Black%20pudding   
38   69    https://simple.wikipedia.org/wiki/Boot%20device   
39   70             https://simple.wikipedia.org/wiki/Boot   
40   71       https://simple.wikipedia.org/wiki/Bankruptcy   
41   74  https://simple.wikipedia.org/wiki/Breakfast%20...   
42   76          https://simple.wikipedia.org/wiki/Browser   
43   80       https://simple.wikipedia.org/wiki/Beekeeping   
44   83  https://simple.wikipedia.org/wiki/British%20En...   
45   85            https://simple.wikipedia.org/wiki/Being   
46   89          https://simple.wikipedia.org/wiki/Beijing   
47   90           https://simple.wikipedia.org/wiki/Bottle   
48   91            https://simple.wikipedia.org/wiki/Berry   
49   93             https://simple.wikipedia.org/wiki/Boil   
50   94            https://simple.wikipedia.org/wiki/Beard   
51   98            https://simple.wikipedia.org/wiki/Black   
52  100  https://simple.wikipedia.org/wiki/Bubonic%20pl...   

                              title  \
0                             April   
1                            August   
2                               Art   
3                                 A   
4                               Air   
5   Autonomous communities of Spain   
6                       Alan Turing   
7                 Alanis Morissette   
8                 Adobe Illustrator   
9                         Andouille   
10                          Farming   
11                       Arithmetic   
12                         Addition   
13                        Australia   
14                 American English   
15                      Aquacult

### Examine the tokenized sections of each document's text.

In [ ]:
sample_encode = tokenizer.encode(df_wiki_data.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'April',
 b' is',
 b' the',
 b' fourth',
 b' month',
 b' of',
 b' the',
 b' year',
 b' in',
 b' the',
 b' Julian',
 b' and',
 b' Greg',
 b'orian',
 b' calendars',
 b',',
 b' and',
 b' comes',
 b' between',
 b' March',
 b' and',
 b' May',
 b'.',
 b' It',
 b' is',
 b' one',
 b' of',
 b' four',
 b' months',
 b' to',
 b' have',
 b' ',
 b'30',
 b' days',
 b'.',
 b' April',
 b' always',
 b' begins',
 b' on',
 b' the',
 b' same',
 b' day',
 b' of',
 b' week',
 b' as',
 b' July',
 b',',
 b' and',
 b' additionally',
 b',',
 b' January',
 b' in',
 b' leap',
 b' years',
 b'.',
 b' April',
 b' always',
 b' ends',
 b' on',
 b' the',
 b' same',
 b' day',
 b' of',
 b' the',
 b' week',
 b' as',
 b' December',
 b'.',
 b' April',
 b"'s",
 b' flowers',
 b' are',
 b' the',
 b' Sweet',
 b' Pe',
 b'a',
 b' and',
 b' Daisy',
 b'.',
 b' Its',
 b' birth',
 b'stone',
 b' is',
 b' the',
 b' diamond',
 b'.',
 b' The',
 b' meaning',
 b' of',
 b' the',
 b' diamond',
 b' is',
 b' innocence',
 b'.',
 b' The',
 b' M

In [6]:
len(decode)

3902

Generate vector data for the text using the Embedding API and add it as a new column named `content_vector`.

In [7]:
def generate_embeddings(text, model=deployment_embedding_name):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
df_wiki_data['content_vector'] = df_wiki_data["text"].apply(lambda x : generate_embeddings (x, model = deployment_embedding_name)) 
df_wiki_data

id                                                url  \
0     1            https://simple.wikipedia.org/wiki/April   
1     2           https://simple.wikipedia.org/wiki/August   
2     6              https://simple.wikipedia.org/wiki/Art   
3     8                https://simple.wikipedia.org/wiki/A   
4     9              https://simple.wikipedia.org/wiki/Air   
5    12  https://simple.wikipedia.org/wiki/Autonomous%2...   
6    13    https://simple.wikipedia.org/wiki/Alan%20Turing   
7    14  https://simple.wikipedia.org/wiki/Alanis%20Mor...   
8    17  https://simple.wikipedia.org/wiki/Adobe%20Illu...   
9    18        https://simple.wikipedia.org/wiki/Andouille   
10   19          https://simple.wikipedia.org/wiki/Farming   
11   21       https://simple.wikipedia.org/wiki/Arithmetic   
12   22         https://simple.wikipedia.org/wiki/Addition   
13   27        https://simple.wikipedia.org/wiki/Australia   
14   28  https://simple.wikipedia.org/wiki/American%20E...   
15   30      https://simple.wikipedia.org/wiki/Aquaculture   
16   32     https://simple.wikipedia.org/wiki/Abbreviation   
17   33            https://simple.wikipedia.org/wiki/Angel   
18   35     https://simple.wikipedia.org/wiki/Ad%20hominem   
19   37  https://simple.wikipedia.org/wiki/Native%20Ame...   
20   39            https://simple.wikipedia.org/wiki/Apple   
21   43  https://simple.wikipedia.org/wiki/Abrahamic%20...   
22   45          https://simple.wikipedia.org/wiki/Algebra   
23   47             https://simple.wikipedia.org/wiki/Atom   
24   48        https://simple.wikipedia.org/wiki/Astronomy   
25   49     https://simple.wikipedia.org/wiki/Architecture   
26   50          https://simple.wikipedia.org/wiki/Anatomy   
27   51         https://simple.wikipedia.org/wiki/Asteroid   
28   52      https://simple.wikipedia.org/wiki/Afghanistan   
29   53           https://simple.wikipedia.org/wiki/Angola   
30   54        https://simple.wikipedia.org/wiki/Argentina   
31   55          https://simple.wikipedia.org/wiki/Austria   
32   56          https://simple.wikipedia.org/wiki/Armenia   
33   57      https://simple.wikipedia.org/wiki/Archaeology   
34   58      https://simple.wikipedia.org/wiki/Application   
35   62           https://simple.wikipedia.org/wiki/Animal   
36   63     https://simple.wikipedia.org/wiki/Acceleration   
37   68  https://simple.wikipedia.org/wiki/Black%20pudding   
38   69    https://simple.wikipedia.org/wiki/Boot%20device   
39   70             https://simple.wikipedia.org/wiki/Boot   
40   71       https://simple.wikipedia.org/wiki/Bankruptcy   
41   74  https://simple.wikipedia.org/wiki/Breakfast%20...   
42   76          https://simple.wikipedia.org/wiki/Browser   
43   80       https://simple.wikipedia.org/wiki/Beekeeping   
44   83  https://simple.wikipedia.org/wiki/British%20En...   
45   85            https://simple.wikipedia.org/wiki/Being   
46   89          https://simple.wikipedia.org/wiki/Beijing   
47   90           https://simple.wikipedia.org/wiki/Bottle   
48   91            https://simple.wikipedia.org/wiki/Berry   
49   93             https://simple.wikipedia.org/wiki/Boil   
50   94            https://simple.wikipedia.org/wiki/Beard   
51   98            https://simple.wikipedia.org/wiki/Black   
52  100  https://simple.wikipedia.org/wiki/Bubonic%20pl...   

                              title  \
0                             April   
1                            August   
2                               Art   
3                                 A   
4                               Air   
5   Autonomous communities of Spain   
6                       Alan Turing   
7                 Alanis Morissette   
8                 Adobe Illustrator   
9                         Andouille   
10                          Farming   
11                       Arithmetic   
12                         Addition   
13                        Australia   
14                 American English   
15                      Aquacult

In [8]:
# Save the data to a CSV file(data/wiki_data_embeddings_3_large.csv)
df_wiki_data.to_csv(os.path.join(os.getcwd(),'data/wiki_data_embeddings_3_large.csv'), index=False)

Analyze the query results to identify relationships based on similarity.

In [9]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model=deployment_embedding_name): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model=deployment_embedding_name # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.content_vector.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_wiki_data, "Tell me about April.", top_n=4)
res = search_docs(df_wiki_data, "Classify the types of art.", top_n=4)
res = search_docs(df_wiki_data, "Draw a table comparing April and August.", top_n=4)

id                                             url         title  \
0    1         https://simple.wikipedia.org/wiki/April         April   
1    2        https://simple.wikipedia.org/wiki/August        August   
3    8             https://simple.wikipedia.org/wiki/A             A   
16  32  https://simple.wikipedia.org/wiki/Abbreviation  Abbreviation   

                                                 text  n_tokens  \
0   April is the fourth month of the year in the J...      3902   
1   August (Aug.) is the eighth month of the year ...      2179   
3   A or a is the first letter of the English alph...       401   
16  An abbreviation is a shorter way to write a wo...       365   

                                       content_vector  similarities  
0   [0.004067655652761459, -0.002844500122591853, ...      0.456704  
1   [0.005639874842017889, -0.010014292784035206, ...      0.246201  
3   [-0.008293329738080502, -0.01762649603188038, ...      0.163401  
16  [-0.0016603072872385383, 0.00935384351760149, ...      0.151688

id                                             url         title  \
2    6           https://simple.wikipedia.org/wiki/Art           Art   
25  49  https://simple.wikipedia.org/wiki/Architecture  Architecture   
33  57   https://simple.wikipedia.org/wiki/Archaeology   Archaeology   
17  33         https://simple.wikipedia.org/wiki/Angel         Angel   

                                                 text  n_tokens  \
2   Art is a creative activity that expresses imag...      1149   
25  Architecture is designing the structures of bu...      1017   
33  Archaeology, or archeology, is the study of th...       872   
17  In many mythologies and religions, an angel is...      1455   

                                       content_vector  similarities  
2   [0.0059301890432834625, 0.003919691313058138, ...      0.536600  
25  [-0.00010834706336027011, 0.003430503187701106...      0.298108  
33  [0.0030919311102479696, -0.013002167455852032,...      0.222888  
17  [-0.01719103381037712, -0.04939764365553856, -...      0.201375

id                                         url     title  \
1    2    https://simple.wikipedia.org/wiki/August    August   
0    1     https://simple.wikipedia.org/wiki/April     April   
12  22  https://simple.wikipedia.org/wiki/Addition  Addition   
3    8         https://simple.wikipedia.org/wiki/A         A   

                                                 text  n_tokens  \
1   August (Aug.) is the eighth month of the year ...      2179   
0   April is the fourth month of the year in the J...      3902   
12  In mathematics, addition, represented by the s...       801   
3   A or a is the first letter of the English alph...       401   

                                       content_vector  similarities  
1   [0.005639874842017889, -0.010014292784035206, ...      0.390042  
0   [0.004067655652761459, -0.002844500122591853, ...      0.362131  
12  [0.01080403570085764, 0.0018525621853768826, -...      0.223811  
3   [-0.008293329738080502, -0.01762649603188038, ...      0.173751

In [10]:
# A function for generating RAG-based answers to user queries.
def generate_rag_answer(user_query, top_n=3):
    content_msg = ""
    res = search_docs(df_wiki_data, user_query, top_n=top_n, to_print=False)
    for index, result in res.iterrows():
        # print(result)
        content_msg = content_msg + result.title + ":\n  " + result.text + "  \n"
    system_msg = """You should generate an answer based on the "### Grouding data" message provided below, rather than using any knowledge you have about the user's question. If there is no "### Grouding data" message, "I could not find a context for the answer." You have to answer.  \n\n### Grouding data  \n""" + content_msg
    print (system_msg + "\nQuestion: " + user_query)

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_query},
        ],
        temperature=0.1,
        max_tokens=2000
    )
    return response.choices[0].message.content

# Generate RAG-based answers to user queries:
user_query = """Compare April and August, summarize the differences for each category, and draw a table."""
response = generate_rag_answer(user_query)
print("Question: " + response)

You should generate an answer based on the "### Grouding data" message provided below, rather than using any knowledge you have about the user's question. If there is no "### Grouding data" message, "I could not find a context for the answer." You have to answer.  

### Grouding data  
August:
  August (Aug.) is the eighth month of the year in the Gregorian calendar, coming between July and September. It has 31 days. It is named after the Roman emperor Augustus Caesar. August does not begin on the same day of the week as any other month in common years, but begins on the same day of the week as February in leap years. August always ends on the same day of the week as November. The Month This month was first called Sextilis in Latin, because it was the sixth month in the old Roman calendar. The Roman calendar began in March about 735 BC with Romulus. October was the eighth month. August was the eighth month when January or February were added to the start of the year by King Numa Pompil